In [ ]:
# ============================================================================
# NetworKit-based PageRank Simulation - Multi-Strategy with Checkpoints
# with Automatic checkpoint saving and resume capability
# ============================================================================

!pip install networkit pandas numpy matplotlib seaborn

import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import seaborn as sns
import networkit as nk
import time
import gc
import pickle
from pathlib import Path
from collections import defaultdict
from datetime import datetime

# === BOOSTING CONFIGURATION ===
NUM_BOOSTING_ROUNDS = 10
BRIDGINGS_PER_ROUND = 10
TOTAL_SIMULATIONS = NUM_BOOSTING_ROUNDS * BRIDGINGS_PER_ROUND

# === THREE CONNECTION RANGES ===
CONNECTION_RANGES = [
    (5, 35, "Range_5-35"),
    (35, 65, "Range_35-65"),
    (65, 95, "Range_65-95"),
]

# === THRESHOLD CONFIGURATION ===
NEUTRAL_THRESHOLD = 0.025  # ±0.025% considered neutral

# === SIMULATION PARAMETERS ===
TOTAL_NODES_WWW = 500000
EDGES_PER_NEW_NODE = 3
PAGERANK_TOLERANCE = 1e-6
PAGERANK_DAMPING = 0.80

# === FILE PATHS ===
BASELINE_PATH = "/content/drive/MyDrive/WebKnoGraph/results/link_graph_edges.csv"

COMPARISON_FOLDERS = [
    "/content/drive/MyDrive/WebKnoGraph/results/automatic_led/random_batches/",
    "/content/drive/MyDrive/WebKnoGraph/results/automatic_led/high_batches/",
    "/content/drive/MyDrive/WebKnoGraph/results/automatic_led/folder_batches/",
    "/content/drive/MyDrive/WebKnoGraph/results/automatic_led/mixed_batches/",
    "/content/drive/MyDrive/WebKnoGraph/results/automatic_led/low_batches/",
]

# === CACHE ===
_www_graph_cache = None


# ============================================================================
# CHECKPOINT SYSTEM
# ============================================================================


class CheckpointManager:
    """Manages checkpoint saving and loading for resumable simulations using pickle"""

    def __init__(self, checkpoint_dir):
        self.checkpoint_dir = Path(checkpoint_dir)
        self.checkpoint_dir.mkdir(exist_ok=True, parents=True)
        self.checkpoint_file = self.checkpoint_dir / "simulation_checkpoint.pkl"
        self.completed_combinations = set()
        self.load_checkpoint()

    def load_checkpoint(self):
        """Load existing checkpoint if available"""
        if self.checkpoint_file.exists():
            try:
                with open(self.checkpoint_file, "rb") as f:
                    data = pickle.load(f)
                    self.completed_combinations = data.get("completed", set())
                print(
                    f"✓ Loaded checkpoint: {len(self.completed_combinations)} combinations already completed"
                )
            except Exception as e:
                print(f"⚠ Could not load checkpoint: {e}")
                self.completed_combinations = set()

    def save_checkpoint(self, strategy_name, range_name, min_conn, max_conn):
        """Save checkpoint after completing a strategy-range combination"""
        combination = (strategy_name, range_name, min_conn, max_conn)
        self.completed_combinations.add(combination)

        checkpoint_data = {
            "completed": self.completed_combinations,
            "last_updated": datetime.now().isoformat(),
            "total_completed": len(self.completed_combinations),
        }

        try:
            with open(self.checkpoint_file, "wb") as f:
                pickle.dump(checkpoint_data, f)
            print(f"  ✓ Checkpoint saved ({len(self.completed_combinations)} total)")
        except Exception as e:
            print(f"  ⚠ Could not save checkpoint: {e}")

    def is_completed(self, strategy_name, range_name, min_conn, max_conn):
        """Check if a strategy-range combination has been completed"""
        combination = (strategy_name, range_name, min_conn, max_conn)
        return combination in self.completed_combinations

    def get_progress(self, total_combinations):
        """Get progress statistics"""
        completed = len(self.completed_combinations)
        remaining = total_combinations - completed
        percent = (
            (completed / total_combinations * 100) if total_combinations > 0 else 0
        )
        return {
            "completed": completed,
            "remaining": remaining,
            "total": total_combinations,
            "percent": percent,
        }

    def reset_checkpoint(self):
        """Reset checkpoint (use with caution)"""
        if self.checkpoint_file.exists():
            self.checkpoint_file.unlink()
        self.completed_combinations = set()
        print("✓ Checkpoint reset")


def mount_google_drive():
    """Mount Google Drive in Colab environment"""
    try:
        from google.colab import drive

        drive.mount("/content/drive")
        print("✓ Google Drive mounted successfully!")
        return True
    except:
        print("⚠ Not in Colab - skipping drive mount")
        return False


def load_graph_from_csv_networkit(file_path):
    """Load directed graph from CSV with FROM/TO columns"""
    try:
        df = pd.read_csv(file_path, usecols=["FROM", "TO"])
        df = df.dropna()
        df["FROM"] = df["FROM"].astype(str)
        df["TO"] = df["TO"].astype(str)
    except Exception as e:
        print(f"✗ Error loading {file_path}: {str(e)}")
        return None, None, None

    from_urls = df["FROM"].values
    to_urls = df["TO"].values

    if len(from_urls) == 0:
        return None, None, None

    all_urls = np.unique(np.concatenate([from_urls, to_urls]))
    url_to_idx = {url: i for i, url in enumerate(all_urls)}

    g = nk.Graph(n=len(all_urls), weighted=False, directed=True)
    for src_url, tgt_url in zip(from_urls, to_urls):
        g.addEdge(url_to_idx[src_url], url_to_idx[tgt_url])

    return g, all_urls, url_to_idx


def create_www_graph_networkit(n_nodes, m_edges, seed=42):
    """Create Barabási-Albert graph representing WWW with caching"""
    global _www_graph_cache

    cache_key = (n_nodes, m_edges, seed)
    if _www_graph_cache is not None and _www_graph_cache[0] == cache_key:
        cached_graph = _www_graph_cache[1]
        new_graph = nk.Graph(
            n=cached_graph.numberOfNodes(), weighted=False, directed=True
        )
        for u, v in cached_graph.iterEdges():
            new_graph.addEdge(u, v)
        return new_graph

    nk.setSeed(seed, False)
    generator = nk.generators.BarabasiAlbertGenerator(
        k=m_edges, nMax=n_nodes, n0=m_edges
    )
    www_graph = generator.generate()

    cached_graph = nk.Graph(n=www_graph.numberOfNodes(), weighted=False, directed=True)
    for u, v in www_graph.iterEdges():
        cached_graph.addEdge(u, v)
    _www_graph_cache = (cache_key, cached_graph)

    return www_graph


def process_configuration_networkit(
    www_graph, kalicube_edges, kalicube_nodes, min_connections, max_connections
):
    """Merge Kalicube graph with WWW and calculate PageRank with specified connection range"""
    kalicube_offset = www_graph.numberOfNodes()
    n_kalicube = len(kalicube_nodes)

    merged_graph = nk.Graph(n=www_graph.numberOfNodes(), weighted=False, directed=True)
    for u, v in www_graph.iterEdges():
        merged_graph.addEdge(u, v)

    for _ in range(n_kalicube):
        merged_graph.addNode()

    if kalicube_edges:
        for src, tgt in kalicube_edges:
            merged_graph.addEdge(src + kalicube_offset, tgt + kalicube_offset)

    n_connections = np.random.randint(min_connections, max_connections + 1)
    n_www_sample = min(n_connections, TOTAL_NODES_WWW)
    n_kalicube_sample = min(n_connections, len(kalicube_nodes))

    www_nodes_sample = np.random.choice(
        TOTAL_NODES_WWW, size=n_www_sample, replace=False
    )
    kalicube_indices = np.random.choice(
        len(kalicube_nodes), size=n_kalicube_sample, replace=False
    )

    for www_node_id, kalicube_idx in zip(www_nodes_sample, kalicube_indices):
        merged_graph.addEdge(www_node_id, kalicube_idx + kalicube_offset)

    pagerank_algo = nk.centrality.PageRank(
        merged_graph, damp=PAGERANK_DAMPING, tol=PAGERANK_TOLERANCE
    )
    pagerank_algo.run()
    pagerank_scores = pagerank_algo.scores()

    pagerank_dict = {}
    for i, url in enumerate(kalicube_nodes):
        pagerank_dict[url] = pagerank_scores[i + kalicube_offset]

    return pagerank_dict


def classify_delta(delta_pct):
    """Classify delta as positive, negative, or neutral"""
    if delta_pct > NEUTRAL_THRESHOLD:
        return "positive"
    elif delta_pct < -NEUTRAL_THRESHOLD:
        return "negative"
    else:
        return "neutral"


def run_boosting_round(
    round_id,
    old_edges,
    new_edges,
    old_nodes,
    new_nodes,
    page_deltas_tracker,
    min_conn,
    max_conn,
):
    """Run one boosting round with multiple bridging simulations"""
    delta_pcts_all = []

    for bridging_id in range(BRIDGINGS_PER_ROUND):
        sim_id = round_id * BRIDGINGS_PER_ROUND + bridging_id
        sim_seed = 42 + sim_id

        np.random.seed(sim_seed)
        random.seed(sim_seed)

        www_graph = create_www_graph_networkit(
            TOTAL_NODES_WWW, EDGES_PER_NEW_NODE, sim_seed
        )

        pagerank_old = process_configuration_networkit(
            www_graph, old_edges, old_nodes, min_conn, max_conn
        )
        pagerank_new = process_configuration_networkit(
            www_graph, new_edges, new_nodes, min_conn, max_conn
        )

        common_urls = set(pagerank_old.keys()) & set(pagerank_new.keys())
        if not common_urls:
            continue

        for url in common_urls:
            old_val = pagerank_old[url]
            new_val = pagerank_new[url]
            delta = new_val - old_val
            delta_pct = (delta / max(old_val, 1e-10)) * 100
            delta_pcts_all.append(delta_pct)
            page_deltas_tracker[url].append(delta_pct)

    return delta_pcts_all, page_deltas_tracker


def run_boosted_comparison(
    baseline_data, comparison_file, min_conn, max_conn, range_name
):
    """Run complete boosted comparison with specific connection range"""
    comparison_name = comparison_file.stem
    print(f"  Processing: {comparison_name} @ {range_name}...", end=" ", flush=True)

    start_time = time.time()

    g_old, nodes_old, _ = baseline_data
    g_new, nodes_new, _ = load_graph_from_csv_networkit(comparison_file)

    if g_new is None:
        print(f"✗ Failed")
        return None

    old_edges = [(u, v) for u, v in g_old.iterEdges()]
    new_edges = [(u, v) for u, v in g_new.iterEdges()]
    old_nodes = nodes_old
    new_nodes = nodes_new

    del g_new
    gc.collect()

    page_deltas_tracker = defaultdict(list)
    all_deltas = []

    for round_id in range(NUM_BOOSTING_ROUNDS):
        round_deltas, page_deltas_tracker = run_boosting_round(
            round_id,
            old_edges,
            new_edges,
            old_nodes,
            new_nodes,
            page_deltas_tracker,
            min_conn,
            max_conn,
        )
        all_deltas.extend(round_deltas)

    if not all_deltas:
        print("✗ No results")
        return None

    pages_positive = 0
    pages_negative = 0
    pages_neutral = 0

    for url, deltas in page_deltas_tracker.items():
        avg_delta = np.mean(deltas)
        classification = classify_delta(avg_delta)

        if classification == "positive":
            pages_positive += 1
        elif classification == "negative":
            pages_negative += 1
        else:
            pages_neutral += 1

    total_pages = len(page_deltas_tracker)
    all_deltas = np.array(all_deltas)

    final_mean = np.mean(all_deltas)
    final_max = np.max(all_deltas)
    final_min = np.min(all_deltas)
    final_std = np.std(all_deltas)

    duration = time.time() - start_time
    print(f"✓ Mean: {final_mean:+.3f}% [{duration:.1f}s]")

    return {
        "name": comparison_name,
        "range_name": range_name,
        "min_connections": min_conn,
        "max_connections": max_conn,
        "duration": duration,
        "mean": final_mean,
        "max": final_max,
        "min": final_min,
        "std": final_std,
        "pages_up": pages_positive,
        "pages_down": pages_negative,
        "pages_neutral": pages_neutral,
        "total_pages": total_pages,
        "num_simulations": TOTAL_SIMULATIONS,
    }


def create_strategy_summary(all_results, output_folder, strategy_name, range_name):
    """Create summary for a single strategy at a specific range"""
    print(f"\n{'=' * 70}")
    print(f"STRATEGY SUMMARY: {strategy_name} @ {range_name}")
    print(f"{'=' * 70}")

    data = []
    for r in all_results:
        if r:
            data.append(
                {
                    "Comparison": r["name"],
                    "Range": r["range_name"],
                    "Min_Conn": r["min_connections"],
                    "Max_Conn": r["max_connections"],
                    "Mean_Delta_%": r["mean"],
                    "Max_Delta_%": r["max"],
                    "Min_Delta_%": r["min"],
                    "Std_Delta_%": r["std"],
                    "Pages_Up": r["pages_up"],
                    "Pages_Down": r["pages_down"],
                    "Pages_Neutral": r["pages_neutral"],
                    "Total_Pages": r["total_pages"],
                    "Duration_m": r["duration"] / 60,
                }
            )

    if not data:
        print("✗ No valid results")
        return None

    df = pd.DataFrame(data)
    df = df.sort_values("Mean_Delta_%", ascending=False)

    # Format strategy name to Title_Case
    formatted_strategy = "_".join(
        word.capitalize() for word in strategy_name.split("_")
    )

    summary_path = output_folder / f"{range_name}_{formatted_strategy}.csv"
    df.to_csv(summary_path, index=False)

    print("\nRankings by Mean Delta %:")
    for idx, row in df.iterrows():
        symbol = (
            "↑"
            if row["Mean_Delta_%"] > NEUTRAL_THRESHOLD
            else "↓"
            if row["Mean_Delta_%"] < -NEUTRAL_THRESHOLD
            else "→"
        )
        print(
            f"  {symbol} {row['Comparison']}: {row['Mean_Delta_%']:+.3f}% "
            + f"(↑{row['Pages_Up']:.0f} ↓{row['Pages_Down']:.0f} →{row['Pages_Neutral']:.0f})"
        )

    strategy_avg_mean = df["Mean_Delta_%"].mean()
    strategy_avg_max = df["Max_Delta_%"].mean()
    strategy_avg_min = df["Min_Delta_%"].mean()
    strategy_avg_up = df["Pages_Up"].mean()
    strategy_avg_down = df["Pages_Down"].mean()
    strategy_avg_neutral = df["Pages_Neutral"].mean()

    print(f"\nStrategy Averages for {range_name}:")
    print(f"  Avg Mean Delta:   {strategy_avg_mean:+.3f}%")
    print(f"  Avg Max Delta:    {strategy_avg_max:+.3f}%")
    print(f"  Avg Min Delta:    {strategy_avg_min:+.3f}%")
    print(f"  Avg Pages Up:     {strategy_avg_up:.1f}")
    print(f"  Avg Pages Down:   {strategy_avg_down:.1f}")
    print(f"  Avg Pages Neutral: {strategy_avg_neutral:.1f}")
    print(f"\n✓ Saved: {summary_path.name}")

    return {
        "strategy_name": strategy_name,
        "range_name": range_name,
        "min_connections": data[0]["Min_Conn"],
        "max_connections": data[0]["Max_Conn"],
        "avg_mean": strategy_avg_mean,
        "avg_max": strategy_avg_max,
        "avg_min": strategy_avg_min,
        "avg_up": strategy_avg_up,
        "avg_down": strategy_avg_down,
        "avg_neutral": strategy_avg_neutral,
        "num_comparisons": len(data),
    }


def update_overall_tracker(overall_tracker_path, strategy_result):
    """Update and save overall tracker"""
    if overall_tracker_path.exists():
        tracker_df = pd.read_csv(overall_tracker_path)
        existing_data = tracker_df.to_dict("records")
    else:
        existing_data = []

    existing_data.append(
        {
            "Strategy": strategy_result["strategy_name"],
            "Range": strategy_result["range_name"],
            "Min_Connections": strategy_result["min_connections"],
            "Max_Connections": strategy_result["max_connections"],
            "Overall_Avg_Mean_%": strategy_result["avg_mean"],
            "Overall_Avg_Max_%": strategy_result["avg_max"],
            "Overall_Avg_Min_%": strategy_result["avg_min"],
            "Overall_Avg_Pages_Up": strategy_result["avg_up"],
            "Overall_Avg_Pages_Down": strategy_result["avg_down"],
            "Overall_Avg_Pages_Neutral": strategy_result["avg_neutral"],
            "Num_Comparisons": strategy_result["num_comparisons"],
        }
    )

    tracker_df = pd.DataFrame(existing_data)
    tracker_df.to_csv(overall_tracker_path, index=False)

    print(f"\n{'=' * 70}")
    print(
        f"OVERALL AVERAGES: {strategy_result['strategy_name']} @ {strategy_result['range_name']}"
    )
    print(f"{'=' * 70}")
    print(
        f"Connection Range:        {strategy_result['min_connections']}-{strategy_result['max_connections']}"
    )
    print(f"Comparison Files:        {strategy_result['num_comparisons']}")
    print(
        f"Total Simulations:       {strategy_result['num_comparisons'] * TOTAL_SIMULATIONS:,}"
    )
    print(f"\nOverall Averages:")
    print(f"  Overall Avg Mean:        {strategy_result['avg_mean']:+.3f}%")
    print(f"  Overall Avg Max:         {strategy_result['avg_max']:+.3f}%")
    print(f"  Overall Avg Min:         {strategy_result['avg_min']:+.3f}%")
    print(f"  Overall Avg Pages Up:    {strategy_result['avg_up']:.1f}")
    print(f"  Overall Avg Pages Down:  {strategy_result['avg_down']:.1f}")
    print(f"  Overall Avg Pages Neutral: {strategy_result['avg_neutral']:.1f}")
    print(f"\n✓ Saved to: {overall_tracker_path.name}")
    print(f"{'=' * 70}")


def process_strategy_with_range(
    baseline_data,
    comparison_folder,
    strategy_name,
    min_conn,
    max_conn,
    range_name,
    main_output_folder,
    overall_tracker_path,
    checkpoint_manager,
):
    """Process a single strategy folder with specific connection range"""

    # Check if already completed
    if checkpoint_manager.is_completed(strategy_name, range_name, min_conn, max_conn):
        print(f"\n⏭ SKIPPING (already completed): {strategy_name} @ {range_name}")
        return None

    comparison_folder = Path(comparison_folder)

    print(f"\n\n{'#' * 70}")
    print(f"STRATEGY: {strategy_name} | RANGE: {range_name} ({min_conn}-{max_conn})")
    print(f"{'#' * 70}")

    comparison_files = list(comparison_folder.glob("*.csv"))

    if not comparison_files:
        print(f"✗ No CSV files in: {comparison_folder}")
        return None

    print(f"✓ Found {len(comparison_files)} comparison files")

    all_results = []
    for i, comp_file in enumerate(comparison_files, 1):
        result = run_boosted_comparison(
            baseline_data, comp_file, min_conn, max_conn, range_name
        )
        all_results.append(result)

        global _www_graph_cache
        _www_graph_cache = None
        gc.collect()

    strategy_result = create_strategy_summary(
        all_results, main_output_folder, strategy_name, range_name
    )

    if strategy_result:
        update_overall_tracker(overall_tracker_path, strategy_result)
        # Save checkpoint after successful completion
        checkpoint_manager.save_checkpoint(
            strategy_name, range_name, min_conn, max_conn
        )

    return strategy_result


if __name__ == "__main__":
    print("=" * 70)
    print("NetworKit Multi-Range BA PageRank Simulation WITH CHECKPOINTS")
    print("=" * 70)
    print(f"NetworKit version: {nk.__version__}")
    print(f"Boosting Rounds: {NUM_BOOSTING_ROUNDS}")
    print(f"Bridgings per Round: {BRIDGINGS_PER_ROUND}")
    print(f"Total Simulations per Comparison: {TOTAL_SIMULATIONS}")

    print(f"\n{'CONNECTION RANGES':=^70}")
    for min_c, max_c, name in CONNECTION_RANGES:
        print(f"  {name}: {min_c}-{max_c} connections")
    print("=" * 70)

    print(f"\n{'SEO OPTIMIZATION PARAMETERS':=^70}")
    print(f"PageRank Damping Factor: {PAGERANK_DAMPING}")
    print(f"PageRank Tolerance: {PAGERANK_TOLERANCE}")
    print(f"Neutral Threshold: ±{NEUTRAL_THRESHOLD}%")
    print("=" * 70)

    total_runs = len(COMPARISON_FOLDERS) * len(CONNECTION_RANGES)
    print(f"\nStrategies: {len(COMPARISON_FOLDERS)}")
    print(f"Connection Ranges: {len(CONNECTION_RANGES)}")
    print(f"Total Strategy-Range Combinations: {total_runs}")

    mount_google_drive()

    baseline_path = Path(BASELINE_PATH)
    if not baseline_path.exists():
        print(f"\n✗ Baseline not found: {BASELINE_PATH}")
        exit(1)

    print(f"\n✓ Baseline: {baseline_path.name}")

    print("\nLoading baseline graph...")
    baseline_data = load_graph_from_csv_networkit(baseline_path)
    if baseline_data[0] is None:
        print("✗ Failed to load baseline")
        exit(1)

    g, nodes, _ = baseline_data
    print(f"✓ Loaded: {g.numberOfNodes():,} nodes, {g.numberOfEdges():,} edges")

    main_output_folder = Path(COMPARISON_FOLDERS[0]).parent / "bar_results_automatic"
    main_output_folder.mkdir(exist_ok=True, parents=True)
    print(f"✓ Output folder: {main_output_folder}")

    # Initialize checkpoint manager
    checkpoint_manager = CheckpointManager(main_output_folder / "checkpoints")

    # Show progress
    progress = checkpoint_manager.get_progress(total_runs)
    print(f"\n{'=' * 70}")
    print(f"CHECKPOINT STATUS")
    print(f"{'=' * 70}")
    print(
        f"Completed: {progress['completed']}/{progress['total']} ({progress['percent']:.1f}%)"
    )
    print(f"Remaining: {progress['remaining']}")
    print(f"{'=' * 70}")

    overall_tracker_path = main_output_folder / "OVERALL_AVERAGES_TRACKER.csv"

    all_strategy_results = []
    run_count = 0

    for min_conn, max_conn, range_name in CONNECTION_RANGES:
        print(f"\n\n{'█' * 70}")
        print(f"CONNECTION RANGE: {range_name} ({min_conn}-{max_conn})")
        print(f"{'█' * 70}")

        for folder in COMPARISON_FOLDERS:
            run_count += 1
            strategy_name = Path(folder).name

            print(f"\n[RUN {run_count}/{total_runs}]")

            strategy_result = process_strategy_with_range(
                baseline_data,
                folder,
                strategy_name,
                min_conn,
                max_conn,
                range_name,
                main_output_folder,
                overall_tracker_path,
                checkpoint_manager,
            )

            if strategy_result:
                all_strategy_results.append(strategy_result)

    print(f"\n\n{'=' * 70}")
    print("✓ ALL STRATEGY-RANGE COMBINATIONS PROCESSED")
    print(f"{'=' * 70}")
    print(f"Combinations Completed: {len(all_strategy_results)}/{total_runs}")
    print(f"\nFinal output files:")
    print(f"  • Strategy-range summaries: {len(all_strategy_results)} CSV files")
    print(f"    Format: <Range>_<Strategy_Name>.csv")
    print(f"    Example: Range_5-35_Folder_Batches.csv")
    print(f"  • Overall tracker: 1 CSV file")
    print(f"  • Checkpoint file: 1 pickle file (.pkl)")
    print(f"\nSaved to: {main_output_folder}")

    if all_strategy_results:
        print(f"\n{'=' * 70}")
        print("BEST PERFORMERS BY RANGE")
        print(f"{'=' * 70}")

        for min_conn, max_conn, range_name in CONNECTION_RANGES:
            range_results = [
                r for r in all_strategy_results if r["range_name"] == range_name
            ]
            if range_results:
                best = max(range_results, key=lambda x: x["avg_mean"])
                print(f"\n{range_name} ({min_conn}-{max_conn}):")
                print(f"  🏆 Best: {best['strategy_name']}")
                print(f"     Mean Δ: {best['avg_mean']:+.3f}%")
                print(f"     ↑Pages: {best['avg_up']:.1f}")

    print(f"\n{'=' * 70}")

NetworKit Multi-Range BA PageRank Simulation WITH CHECKPOINTS
NetworKit version: 11.1.post1
Boosting Rounds: 10
Bridgings per Round: 10
Total Simulations per Comparison: 100

==========================CONNECTION RANGES===========================
  Range_5-35: 5-35 connections
  Range_35-65: 35-65 connections
  Range_65-95: 65-95 connections

=====================SEO OPTIMIZATION PARAMETERS======================
PageRank Damping Factor: 0.8
PageRank Tolerance: 1e-06
Neutral Threshold: ±0.025%

Strategies: 5
Connection Ranges: 3
Total Strategy-Range Combinations: 15
Mounted at /content/drive
✓ Google Drive mounted successfully!

✓ Baseline: link_graph_edges.csv

Loading baseline graph...
✓ Loaded: 1,841 nodes, 122,066 edges
✓ Output folder: /content/drive/MyDrive/WebKnoGraph/results/automatic_led/bar_results_automatic

CHECKPOINT STATUS
Completed: 0/15 (0.0%)
Remaining: 15


██████████████████████████████████████████████████████████████████████
CONNECTION RANGE: Range_5-35 (5-35)
███████